V8 OVA_Prostate dataset from openml 

V6,7 - reformat code - add compraison with Gauss and uniform datasets with the same size 

V5 - single-cell-rna-seq-from-stoeckius-et-al-2017/GSE100866_CD8_merged-RNA_umi

V4 - GSE67123_v6_scrna_10x_143_23548_Mouse_Embryo_HSCs_invivo_fromCytotrace.csv

V3 - human_liver from ARCHS4

V2 - GSE90047_Mouse_Liver_Hepatoblast_invivo

V1 - Dexter300 dataset 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.neighbors import kneighbors_graph
import time
import matplotlib.pyplot as plt

try:
    import cudf
    from cuml.neighbors import NearestNeighbors
    #from cuml.datasets import make_blobs    
except:
    print('GPU library cannot be imported. Turn ON GPU')
    

# Load Data

In [ ]:
pip install openml


In [ ]:
import openml

t0 = time.time()
openml_df = openml.datasets.list_datasets(output_format="dataframe")
print(time.time()-t0,'seconds passed ')

if 0: ## The same can be done with BIGGER lines of code
    openml_list = openml.datasets.list_datasets()  # returns a dict
    # Show a nice table with some key data properties
    openml_df = pd.DataFrame.from_dict(openml_list, orient="index")
    openml_df = datalist[["did", "name", "NumberOfInstances", "NumberOfFeatures", "NumberOfClasses"]]

    print(f"First 10 of {len(datalist)} datasets...")
    openml_df.head(n=10)


openml_df

# Among all datasets there is subset of 45 datasets with  mircoarray Gene Expressions (mainly cancer) 
# can be selected by number of features == 10937
df_gemler = openml_df[ ( openml_df.NumberOfFeatures == 10937)    ].sort_values(["NumberOfInstances"], ascending=False)#.head(n=20)
print(df_gemler.shape)
df_gemler

In [ ]:
dataset = '' 

if dataset == 'Dexter300from_scikit_hubness':
    f = '/kaggle/input/hubness-for-high-dimensional-datasets/y_Dexter300from_scikit_hubness.csv'
    y = pd.read_csv(f ,  index_col= 0  )
    y
    f = '/kaggle/input/hubness-for-high-dimensional-datasets/X_Dexter300from_scikit_hubness.csv'
    X = pd.read_csv(f ,  index_col= 0  )
    X

if dataset == 'GSE90047_Mouse_Liver_Hepatoblast_invivo':
    f = '/kaggle/input/trajectory-inference-single-cell-rna-seq/GSE90047_Mouse_Liver_Hepatoblast_invivo.csv'
    X = pd.read_csv(f ,  index_col= 0  )
    X

if dataset == 'human_liver':
    f = '/kaggle/input/human-liver-rnaseq-gene-expression-903-samples/human_liver.tsv'
    X = pd.read_csv(f ,  index_col= 0 , sep = '\t' )
    X = X.transpose()
    X

if dataset == 'GSE67123_v6_scrna_10x_143_23548_Mouse_Embryo_HSCs_invivo_fromCytotrace':
    f = '/kaggle/input/genes-expressions-datasets-collection/GSE67123_v6_scrna_10x_143_23548_Mouse_Embryo_HSCs_invivo_fromCytotrace.csv'
    X = pd.read_csv(f ,  index_col= 0 )#  , sep = '\t' )
    X
    
if dataset == 'single-cell-rna-seq-from-stoeckius-et-al-2017/GSE100866_CD8_merged-RNA_umi':
    f = '/kaggle/input/single-cell-rna-seq-from-stoeckius-et-al-2017/GSE100866_CD8_merged-RNA_umi.csv'
    X = pd.read_csv(f ,  index_col= 0 )#  , sep = '\t' )
    X = X.transpose()
    X

dataset = openml.datasets.get_dataset(1146)
X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format="dataframe", target=dataset.default_target_attribute )
X

In [ ]:
from sklearn.decomposition import PCA

t0=time.time()
fig = plt.figure(figsize = (20,4) )

X2 = PCA().fit_transform(X)
plt.scatter(X2[:,0],X2[:,1]    )
plt.title('PCA')
plt.grid()
plt.show()

print(time.time()-t0,'seconds passed')


# Function - degrees of KNN graph

In [ ]:
#distribution_type = 'Gauss' # 'uniform'

def get_vec_degs(X, n_neighbors = 5, method = 'sklearn', verbose = 1 ): 
    #verbose = 0
    #method = 'sklearn'
    #n_neighbors = 5

    t00=time.time()

    if method in ['GPU']:
        X_cudf = cudf.DataFrame(X)
        model = NearestNeighbors(n_neighbors=n_neighbors+1)
        model.fit(X)
        # get 3 nearest neighbors
        m = model.kneighbors_graph(X_cudf)#, include_self=False) #  no include_self
        vec_degs = np.sum(m,axis = 0 )
        vec_degs = vec_degs.get()

        vec_degs = np.squeeze(np.array(vec_degs.ravel()))
        vec_degs -= np.ones_like(vec_degs ) # no include_self - so that is a way round

    else: #if method in ['sklearn']:
        m = kneighbors_graph(X, n_neighbors= n_neighbors, mode='connectivity' , include_self=False) #'distance' mode=  'connectivity'
        vec_degs = np.sum(m,axis = 0 )
        vec_degs = np.squeeze(np.array(vec_degs.ravel()))

    if verbose >= 1:
        print( np.round(time.time()-t00,1),'seconds passed total')    
    
    return vec_degs

# Historgrams for data only 

In [ ]:
t00 = time.time()
df_stat = pd.DataFrame()

for n_neighbors in [5,10]:
    list_vec_degs = []
    list_info = []
    vec_degs = get_vec_degs(X, n_neighbors = n_neighbors, method = 'sklearn', verbose = 1 )
    list_vec_degs.append(vec_degs)
    list_info.append('Data')

    fig = plt.figure(figsize=(20,10))

    fig.add_subplot(2,1,1)
    c = 0
    for vec_degs in list_vec_degs[:1]:
        c += 1
        t0 = time.time()

        bins = np.arange(50)# np.max(vec_degs))
        h = np.histogram(vec_degs,bins = bins, density= True)

        label = ' n_neighbors'+str(n_neighbors) + ' ' + list_info[c-1]
        plt.plot(h[0],'*-',label = label)

        st = pd.Series(vec_degs).describe()
        st.name = label
        df_stat = df_stat.join( st, how = 'outer'  )
    plt.title(' n_neighbors'+str(n_neighbors)  )
    plt.legend()
    plt.grid()


    fig.add_subplot(2,1,2)
    c = 0
    for vec_degs in list_vec_degs[:1]:
        c += 1
        t0 = time.time()

        bins = np.arange(50)# np.max(vec_degs))
        h = np.histogram(vec_degs,bins = bins, density= True)

        label = ' n_neighbors'+str(n_neighbors) + ' ' + list_info[c-1]
        plt.loglog(h[0],'*-',label = label)
    plt.title(' n_neighbors'+str(n_neighbors)  )
    plt.legend()
    plt.grid()

df_stat

# Plot histograms for data and benchamrks

In [ ]:
t00 = time.time()
df_stat = pd.DataFrame()

for n_neighbors in [5,10]:
    
    list_vec_degs = []
    list_info = []
    vec_degs = get_vec_degs(X, n_neighbors = n_neighbors, method = 'sklearn', verbose = 1 )
    list_vec_degs.append(vec_degs)
    list_info.append('Data')

    X2 = np.random.randn(*X.shape)
    vec_degs = get_vec_degs(X2, n_neighbors = n_neighbors, method = 'sklearn', verbose = 1 )
    list_vec_degs.append(vec_degs)
    list_info.append('Gauss')

    X2 = np.random.rand(*X.shape)
    vec_degs = get_vec_degs(X2, n_neighbors = n_neighbors, method = 'sklearn', verbose = 1 )
    list_vec_degs.append(vec_degs)
    list_info.append('Uniform')
    

    fig = plt.figure(figsize=(20,10))

    fig.add_subplot(2,1,1)
    c = 0
    for vec_degs in list_vec_degs:
        c += 1
        t0 = time.time()

        bins = np.arange(50)# np.max(vec_degs))
        h = np.histogram(vec_degs,bins = bins, density= True)

        label = ' n_neighbors'+str(n_neighbors) + ' ' + list_info[c-1]
        plt.plot(h[0],'*-',label = label)

        st = pd.Series(vec_degs).describe()
        st.name = label
        df_stat = df_stat.join( st, how = 'outer'  )
    plt.title(' n_neighbors'+str(n_neighbors)  )
    plt.legend()
    plt.grid()


    fig.add_subplot(2,1,2)
    c = 0
    for vec_degs in list_vec_degs:
        c += 1
        t0 = time.time()

        bins = np.arange(50)# np.max(vec_degs))
        h = np.histogram(vec_degs,bins = bins, density= True)

        label = ' n_neighbors'+str(n_neighbors) + ' ' + list_info[c-1]
        plt.loglog(h[0],'*-',label = label)
    plt.title(' n_neighbors'+str(n_neighbors)  )
    plt.legend()
    plt.grid()

df_stat

# Further comparaison with Gauss and uniform of other dimension

In [ ]:
 list_vec_degs = []
list_info = []
vec_degs = get_vec_degs(X, n_neighbors = 5, method = 'sklearn', verbose = 1 )
list_vec_degs.append(vec_degs)
list_info.append('Data')

X2 = np.random.randn(*X.shape)
vec_degs = get_vec_degs(X2, n_neighbors = 5, method = 'sklearn', verbose = 1 )
list_vec_degs.append(vec_degs)
list_info.append('Gauss')

X2 = np.random.rand(*X.shape)
vec_degs = get_vec_degs(X2, n_neighbors = 5, method = 'sklearn', verbose = 1 )
list_vec_degs.append(vec_degs)
list_info.append('Uniform')

X2 = np.random.randn(X.shape[0], int( X.shape[0]/4 ))
vec_degs = get_vec_degs(X2, n_neighbors = 5, method = 'sklearn', verbose = 1 )
list_vec_degs.append(vec_degs)
list_info.append('Gauss dim->dim/4')

X2 = np.random.rand(X.shape[0], int( X.shape[0]/4 ))
vec_degs = get_vec_degs(X2, n_neighbors = 5, method = 'sklearn', verbose = 1 )
list_vec_degs.append(vec_degs)
list_info.append('Uniform dim->dim/4')

X2.shape

In [ ]:
t00 = time.time()
df_stat = pd.DataFrame()

for n_neighbors in [5]:

    fig = plt.figure(figsize=(20,10))

    #fig.add_subplot(2,1,1)
    c = 0
    for vec_degs in list_vec_degs:
        c += 1
        t0 = time.time()

        bins = np.arange(50)# np.max(vec_degs))
        h = np.histogram(vec_degs,bins = bins, density= True)

        label = ' n_neighbors'+str(n_neighbors) + ' ' + list_info[c-1]
        plt.plot(h[0],'*-',label = label)

        st = pd.Series(vec_degs).describe()
        st.name = label
        df_stat = df_stat.join( st, how = 'outer'  )
    plt.title(' n_neighbors'+str(n_neighbors)  )
    plt.legend()
    plt.grid()


    fig = plt.figure(figsize=(20,10)) # fig.add_subplot(2,1,2)
    c = 0
    for vec_degs in list_vec_degs:
        c += 1
        t0 = time.time()

        bins = np.arange(50)# np.max(vec_degs))
        h = np.histogram(vec_degs,bins = bins, density= True)

        label = ' n_neighbors'+str(n_neighbors) + ' ' + list_info[c-1]
        plt.loglog(h[0],'*-',label = label)
    plt.title(' n_neighbors'+str(n_neighbors)  )
    plt.legend()
    plt.grid()

df_stat